In [11]:
import os
import mailbox
import csv
def extract_mbox(file_path):
    mbox = mailbox.mbox(file_path)
    path_to_csv = file_path.rsplit('/', 1)[0]
    with open(f'{path_to_csv}/emails.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['From', 'To', 'Subject', 'Date', 'Message', 'Attachment_Path']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for message in mbox:
            attachment_paths = []
            for part in message.walk():
                if part.get_content_disposition() == 'attachment':
                    filename = part.get_filename()
                    if filename:
                        # Generate a unique name for the attachment file
                        attachment_name = f"{message['Message-ID'].strip('<>')}_{filename}"
                        # Create the 'attachments' folder if it does not exist
                        if not os.path.exists(f'{path_to_csv}/attachments'):
                            os.mkdir(f'{path_to_csv}/attachments')
                        # Save the attachment file to the 'attachments' folder
                        attachment_path = os.path.join(f'{path_to_csv}/attachments', attachment_name)
                        payload = part.get_payload(decode=True)
                        if payload is not None:
                            with open(attachment_path, 'wb') as f:
                                f.write(payload)
                            attachment_paths.append(attachment_path)

            row = {}
            row['From'] = message['From']
            row['To'] = message['To']
            row['Subject'] = message['Subject']
            row['Date'] = message['Date']
            row['Message'] = message.get_payload()
            row['Attachment_Path'] = ';'.join(attachment_paths)
            writer.writerow(row)


In [12]:
extract_mbox('/home/lv/Downloads/google_takeout/Takeout/Mail/All mail Including Spam and Trash.mbox')
